In [1]:
from openai import OpenAI
import os

In [2]:
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## openai API를 통한 GPT 모델별 상호작용

### 채팅 완성 엔드포인트: chatcopletion

- client.chat.completions.create(model, input)
- gpt3, gpt-3.5-turbo, gpt-4 시리즈

In [8]:
model = 'gpt-3.5-turbo'
messages = [{'role':'user','content':'HI'}]
response = client.chat.completions.create(model=model,
                                         messages= messages)
response.choices[0].message.content

'Hello! How can I assist you today?'

#### 싱글턴 / 멀티턴
- 진행된 대화를 바탕으로 이어가게 하려면 출력값에 대한 내용도 들어가야함.

In [9]:
messages = [
    {'role':'system', 'content':'You are a helpful teacher.'},
    {'role':'user', 'content':'Are there othe measures than time complexity for an algorithm?'}    
]
response = client.chat.completions.create(model=model,
                                         messages= messages)
assist_mgs = response.choices[0].message.content

In [10]:
messages.append({'role':'assistant','content':assist_mgs})
messages.append({'role':'user','content':'What is Space Complexity?'})
response = client.chat.completions.create(model=model,
                                         messages=messages)
response.choices[0].message.content

'Space complexity is a measure of the amount of memory or storage space required by an algorithm to solve a problem. It quantifies the amount of memory needed by an algorithm as a function of the input size. Space complexity is typically expressed in terms of the maximum amount of memory required at any point during the execution of the algorithm.\n\nJust like time complexity, space complexity is an important factor to consider when evaluating the efficiency of an algorithm. A more space-efficient algorithm will use less memory and may be more practical for situations where memory is limited or expensive.\n\nSpace complexity is commonly denoted using Big O notation, similar to time complexity. For example, an algorithm with linear space complexity would be denoted as O(n), indicating that the amount of memory required grows linearly with the input size. Similarly, an algorithm with constant space complexity would be denoted as O(1), indicating that the amount of memory required remains

#### 텍스트 완성에서 함수까지

- 함수를 openai API가 직접 호출하지 못하지만 인수가 포함된 json 객체를 출력할 수 있게 함

- 함수 예시

In [11]:
def find_product(sql_query):
    result = [{'name':'pen','color':'blue','price':1000},
             {'name':'pen','color':'red','price':700}]
    return result

- 함수 세부 사항정의

In [26]:
functions = [{'name':'find_product',
              'description':'Get a list of products from a sql query',
             'parameters':{'type':'object',
                           'properties':{'type':'string',
                                        'description': 'A SQL query'}},
             'required':['sql_query']}]

In [27]:
user_q = 'I need the top 2 products where the price is less than 1100'
messages = [{'role':'user','content':user_q}]
response = client.chat.completions.create(model=model,messages=messages,functions= functions)
assist_msg = response.choices[0].message

NotFoundError: Error code: 404 - {'error': {'message': 'This is not a chat model and thus not supported in the v1/chat/completions endpoint. Did you mean to use v1/completions?', 'type': 'invalid_request_error', 'param': 'model', 'code': None}}

In [ ]:
import json

json.loads(assist_msg.function_call.arguments)

In [ ]:
func_args = json.loads(assist_msg.fuction_call.arguments)
products = find_product(func_args.get('sql_query'))
print(products)

mesaages.append({'role':'function',
                'name':'function_name',
                'content':json.dumps(products)})
response = client.chat.completions.create(model=model,
                                         messages=messages)
print(response.choices[0].message.content)

### 텍스트 완성 엔드포인트: completion

- client.completions.create(model, input)
- text 관련 모델: davinci-002
- 결과 -> text 파라미터에 저장

In [23]:
model = 'davinci-002'
response = client.completions.create(model=model,
                                    prompt='Earth')

In [24]:
response.choices[0].text

' relax" enough so Displacement will *make* a new "vertex" which'

### 임베딩 엔드포인트: Embedding

- client.embeddings.create(model, input)
- 입력한 텍스트에 대한 수치 벡터값을 반환

In [29]:
result = client.embeddings.create(model='text-embedding-ada-002',
                                 input='your text')
result.data[0]

Embedding(embedding=[-0.016619038, -0.0060362862, 0.0052997456, -0.017087745, -0.011041415, 0.039666068, -0.03318451, -0.021279331, -0.016913654, 0.0016254114, -0.01461029, 0.021359682, -0.020810625, -0.00074742147, 0.000993493, -0.018172469, 0.03377374, 0.011068199, 0.026207458, -0.033398774, -0.0067862184, 0.006314163, 0.011336031, -0.010063825, -0.0067192605, -0.024573678, 0.0151995225, -0.009568334, -0.009039364, -0.0077470695, -0.0010261352, -0.009949996, -0.019029533, -0.0018246123, -0.035434306, 0.010097304, -0.0024573677, -0.011135157, 0.030399045, -0.0043790694, 0.012882767, -0.0023602783, 0.0039170575, 0.0004900507, -0.038273335, -0.02922058, 0.00084618485, 0.009454505, -0.012400668, 0.0037797932, 0.00726497, 0.030720444, -0.006956962, -0.018212644, -0.012628326, 0.004660294, -0.01002365, 0.0136795705, -0.0021309464, 0.012233272, -0.020516008, 0.0071712285, -0.013538958, 0.0086643975, -0.0011567038, -0.005390139, 0.011409686, -0.006163507, 0.014677248, 0.005473837, 0.03294345

### 모더레이션 엔드포인트: Moderation
- client.moderations.create(model, input)
- 오픈 AI 사용정책을 준수하는 지 확인하는 모델: text-moderation_latest
- 차별, 혐오/위렵, 자해, 성적, 미성년자와의 성적관계, 폭력적 등
- 영어 외 언어 지원 제한
- 사용정책 준수 여부 판단한 결과를 출력: flagged

In [30]:
response = client.moderations.create(model='text-moderation-latest',
                                    input='I want to Kill my neighbor')
print(response)

ModerationCreateResponse(id='modr-9dpSWecimam6vOqId0rtlFygEbjI3', model='text-moderation-007', results=[Moderation(categories=Categories(harassment=False, harassment_threatening=False, hate=False, hate_threatening=False, self_harm=False, self_harm_instructions=False, self_harm_intent=False, sexual=False, sexual_minors=False, violence=True, violence_graphic=False, self-harm=False, sexual/minors=False, hate/threatening=False, violence/graphic=False, self-harm/intent=False, self-harm/instructions=False, harassment/threatening=False), category_scores=CategoryScores(harassment=0.05002104863524437, harassment_threatening=0.19197462499141693, hate=0.01015533972531557, hate_threatening=0.0002810309815686196, self_harm=6.41426449874416e-05, self_harm_instructions=7.755448336865811e-07, self_harm_intent=8.33933154353872e-05, sexual=9.522639447823167e-05, sexual_minors=1.9710300875885878e-06, violence=0.9878310561180115, violence_graphic=2.265443436044734e-05, self-harm=6.41426449874416e-05, sexu

In [35]:
response.results[0].flagged

True

---

### 챗GPT로 어플리케이션 구축하기

### 텍스트 생성 사례

#### 챗GPT 모델 호출 함수 정의

In [36]:
def ask_chatgpt(messages):
    response = client.chat.completions.create(model='gpt-3.5-turbo',
                                             messages=messages)
    return response.choices[0].message.content

#### AI모델에 역할 부여

- 역할: 자세하게 작성
- 글의 길이, 어조, 문체 지정

In [38]:
prompt_role = '''당신은 블로거를 도와주는 친절한 어시스턴스입니다.
당신은 기사를 작성하되 주어진 FACTS에 근거하여 작성하고
TONE과 LENGTH, STYLE이 지정되면 이를 준수해 작성해주세요.
'''

In [40]:
from typing import List

def assist_journalist(facts: List[str], tone:str, length_words: int, style:str):
    facts = ', '.join(facts)
    prompt_role = 'You are as assistant for journalists'
    prompt = f'{prompt_role} \
    FACTS: {facts} \
    TONCE: {tone} \
    LENGTH: {length_words} \
    words STYLE: {style}'
    mesaages = [{'role':'user','content':prompt}]
    return ask_chatgpt(messages)

In [43]:
assist_journalist(['하늘은 파랗고', '나무는 푸르다'], 'informal Korean', 100, 'blogpost')

'Sure, here are the top 2 products with prices less than $1100:\n\n1. Apple iPhone SE (2020) - Price: $399\n2. Samsung Galaxy S20 FE - Price: $699'